In [ ]:
# ──────────────────────────────────────────────────────────
# 0) Environment
# ──────────────────────────────────────────────────────────
!pip install -q -U transformers accelerate peft bitsandbytes pandas scikit-learn tqdm aif360
from google.colab import drive
drive.mount('/content/drive')

# ──────────────────────────────────────────────────────────
# 1)
# ──────────────────────────────────────────────────────────
import os, gc, time, copy, torch, numpy as np, pandas as pd
from tqdm.auto import tqdm
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          TrainingArguments, Trainer)
from peft import LoraConfig, get_peft_model
from datasets import Dataset
from aif360.datasets import BinaryLabelDataset
from sklearn.metrics import (accuracy_score, recall_score, precision_score,
                             f1_score, matthews_corrcoef)
from aif360.metrics import ClassificationMetric

label_field = "Probability"

from transformers import TrainerCallback
from sklearn.preprocessing import MinMaxScaler

class LossThresholdCallback(TrainerCallback):
    """
    early stop。
    """
    def __init__(self, threshold: float = 0.10, patience: int = 5):
        self.threshold = threshold
        self.patience  = patience
        self.counter   = 0          #

    def on_log(self, args, state, control, logs=None, **kwargs):
        loss = logs.get("loss")
        if loss is None:
            return control

        if loss < self.threshold:
            self.counter += 1
        else:
            self.counter  = 0       #

        if self.counter >= self.patience:
            control.should_training_stop = True   #
            control.should_save           = True  #  checkpoint
            print(f"[EarlyStop] loss {loss:.4f} < {self.threshold} "
                  f"for {self.counter} steps; stopping training.")

        return control       # return control


def df_to_prompts_and_responses(df, label_col):
    feat_cols = [c for c in df.columns if c != label_col]
    ps, rs = [], []
    for _, row in df.iterrows():
        feats = "\n".join(f"{c}: {row[c]}" for c in feat_cols)
        prompt = ("Based on the following features, what is the class?\n"
                  f"{feats}\n"
                  "Answer with 0 or 1 only.")
        ps.append(prompt)
        rs.append(str(row[label_col]))
    return ps, rs

def build_dataset(prompts, answers, tokenizer):
    rec = []
    for p, a in zip(prompts, answers):
        pid = tokenizer(p, add_special_tokens=False).input_ids
        aid = tokenizer(a, add_special_tokens=False).input_ids
        rec.append({"input_ids": pid+aid,
                    "attention_mask": [1]*(len(pid)+len(aid)),
                    "labels": [-100]*len(pid)+aid})
    return Dataset.from_list(rec)

def data_collator(batch, pad_id):
    max_len = max(len(x["input_ids"]) for x in batch)
    pad = lambda seq, val: seq + [val]*(max_len-len(seq))
    return {
        "input_ids":     torch.tensor([pad(x["input_ids"], pad_id) for x in batch]),
        "attention_mask":torch.tensor([pad(x["attention_mask"], 0) for x in batch]),
        "labels":        torch.tensor([pad(x["labels"], -100)    for x in batch])
    }

def measure_final_score(dataset_orig_test, dataset_orig_predict,
                        privileged_groups, unprivileged_groups, attr2):
    y_test = dataset_orig_test.labels
    y_pred = dataset_orig_predict.labels
    acc   = accuracy_score (y_test, y_pred)
    rec   = recall_score   (y_test, y_pred, average='macro')
    prec  = precision_score(y_test, y_pred, average='macro')
    f1    = f1_score       (y_test, y_pred, average='macro')
    mcc   = matthews_corrcoef(y_test, y_pred)

    cm = ClassificationMetric(dataset_orig_test, dataset_orig_predict,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups  =privileged_groups)
    spd = abs(cm.statistical_parity_difference())
    aod = abs(cm.average_odds_difference())
    eod = abs(cm.equal_opportunity_difference())

    df_t = dataset_orig_test.convert_to_dataframe()[0]
    df_p = df_t.copy(); df_p[label_field] = y_pred

    # tt3 = BinaryLabelDataset(1,0, df_t, [label_field],[attr2])
    # tt4 = BinaryLabelDataset(1,0, df_p, [label_field],[attr2])
    tt3 = BinaryLabelDataset(
    favorable_label   = 1,
    unfavorable_label = 0,
    df                = df_t,
    label_names       = [label_field],
    protected_attribute_names = [attr2]
    )

    tt4 = BinaryLabelDataset(
        favorable_label   = 1,
        unfavorable_label = 0,
        df                = df_p,
        label_names       = [label_field],
        protected_attribute_names = [attr2]
    )

    cm2 = ClassificationMetric(tt3, tt4,
                               unprivileged_groups=[{attr2:0}],
                               privileged_groups  =[{attr2:1}])
    spd2 = abs(cm2.statistical_parity_difference())
    aod2 = abs(cm2.average_odds_difference())
    eod2 = abs(cm2.equal_opportunity_difference())

    tpr_p = cm.true_positive_rate(privileged=True)
    tpr_u = cm.true_positive_rate(privileged=False)
    tnr_p = cm.true_negative_rate(privileged=True)
    tnr_u = cm.true_negative_rate(privileged=False)

    df_t['y_pred'] = y_pred
    sens = list(privileged_groups[0].keys())[0]
    acc_p = (df_t[df_t[sens]==1][label_field] == df_t[df_t[sens]==1]['y_pred']).mean()
    acc_u = (df_t[df_t[sens]==0][label_field] == df_t[df_t[sens]==0]['y_pred']).mean()

    return (acc, rec, prec, f1, mcc, spd, aod, eod,
            spd2, aod2, eod2, tpr_p, tpr_u, tnr_p, tnr_u, acc_p, acc_u)

# ──────────────────────────────────────────────────────────
# 2) tasks list
# ──────────────────────────────────────────────────────────
# tasks = [
#     ("adult","sex"), ("adult","race"),
#     ("compas","sex"), ("compas","race"),
#     ("default","sex"), ("default","age"),
#     ("mep1","sex"), ("mep1","race"),
#     ("mep2","sex"), ("mep2","race")
# ]
tasks = [
    # ("adult","sex"), ("adult","race"),
    # ("compas","sex"), ("compas","race"),
    # ("default","sex"), ("default","age"),
    ("mep1","sex"), ("mep1","race"),
    ("mep2","sex"), ("mep2","race")
]

versions = ["orig", "cot"]                   #
macro_var = {'adult':['sex','race'],'compas':['sex','race'],
             'default':['sex','age'],'mep1':['sex','race'],'mep2':['sex','race']}
base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
drive_root = "/content/drive/MyDrive/Dataset"

# ──────────────────────────────────────────────────────────
# 3) main loop
# ──────────────────────────────────────────────────────────
scaler = MinMaxScaler()
for ds, attr in tasks:
    log_num = 50 if ds in {"mep1", "mep2"} else 50
    other_attr = [a for a in macro_var[ds] if a != attr][0]  # attr2
    for ver in versions:
        print(f"\n##### {ds}-{attr}-{ver} #####")
        train_csv = f"{drive_root}/z_{ds}_{attr}_{ver}_train.csv"
        test_csv  = f"{drive_root}/z_{ds}_{attr}_test.csv"
        out_dir   = f"{drive_root}/model/TinyLlama_{ds}_{attr}_{ver}_lora"
        os.makedirs(out_dir, exist_ok=True)

        # ---- data ----
        train_df = pd.read_csv(train_csv)
        test_df  = pd.read_csv(test_csv)
        train_p, train_r = df_to_prompts_and_responses(train_df, label_field)
        test_p ,  test_r = df_to_prompts_and_responses(test_df , label_field)



        # ---- model ----
        tok = AutoTokenizer.from_pretrained(base_model)
        tok.pad_token_id = tok.pad_token_id or tok.eos_token_id
        mdl = AutoModelForCausalLM.from_pretrained(base_model, load_in_8bit=True, device_map="auto")
        mdl = get_peft_model(mdl, LoraConfig(r=8,lora_alpha=16,
                                             target_modules=["q_proj","v_proj"],
                                             lora_dropout=0.05,bias="none",
                                             task_type="CAUSAL_LM"))
        tr = Trainer(
            mdl,
            TrainingArguments(out_dir, per_device_train_batch_size=16,
                              num_train_epochs=3, learning_rate=2e-4,
                              fp16=True, logging_steps=log_num,
                              save_strategy="epoch", report_to=[]),
            train_dataset = build_dataset(train_p, train_r, tok),
            data_collator = lambda b: data_collator(b, tok.pad_token_id),
            callbacks       = [LossThresholdCallback(threshold=0.1, patience=1)]
        )
        tr.train(); mdl.save_pretrained(out_dir); tok.save_pretrained(out_dir)
        del tr; gc.collect(); torch.cuda.empty_cache()

        # ---- preview ----
        print("Preview 5:")
        for prm in test_p[:5]:
            g = mdl.generate(**tok(prm,return_tensors="pt").to(mdl.device),
                             max_new_tokens=2, do_sample=False)
            print(tok.decode(g[0], skip_special_tokens=True).split('\n')[-1])

        # ---- batch ----
        preds=[]
        batch_size = 256
        # enc = tok(test_p, padding=True, return_tensors="pt")
        # for s in tqdm(range(0,len(enc["input_ids"]),256)):
        #     e=s+128
        #     out=mdl.generate(input_ids=enc["input_ids"][s:e].to(mdl.device),
        #                      attention_mask=enc["attention_mask"][s:e].to(mdl.device),
        #                      max_new_tokens=3, do_sample=False)
        #     for i in range(out.size(0)):
        #         for tk in out[i, enc["input_ids"].size(1):].tolist():
        #             t=tok.decode(tk).strip()
        #             if t in ("0","1"): preds.append(int(t)); break
        #         else: preds.append(0)
        enc = tok(test_p, padding=True, return_tensors="pt")
        for s in tqdm(range(0, len(enc["input_ids"]), batch_size)):
            e = min(s + batch_size, len(enc["input_ids"]))
            # e = min(s + 128, len(enc["input_ids"]))
            out = mdl.generate(
                input_ids      = enc["input_ids"][s:e].to(mdl.device),
                attention_mask = enc["attention_mask"][s:e].to(mdl.device),
                max_new_tokens = 3, do_sample=False)

            seq_len_batch = enc["input_ids"][s:e].size(1)   #

            for i in range(out.size(0)):
                for tk in out[i, seq_len_batch:].tolist():  #
                    t = tok.decode(tk).strip()
                    if t in ("0", "1"):
                        preds.append(int(t)); break
                else:
                    preds.append(0)


        # ---- save CSV ----
        pd.concat([test_df, pd.Series(preds,name="Predicted")],axis=1)\
          .to_csv(f"{out_dir}/test_predictions.csv",index=False)

        # ---- fairness metrics ----
        ds_test = BinaryLabelDataset(
            favorable_label   = 1,
            unfavorable_label = 0,
            df                = test_df[[label_field, attr, other_attr]],
            label_names       = [label_field],
            protected_attribute_names = [attr, other_attr]
        )

        ds_pred = ds_test.copy(deepcopy=True)
        ds_pred.labels = np.array(preds).reshape(-1, 1)


        res = measure_final_score(ds_test, ds_pred,
                                  [{attr:1}], [{attr:0}], other_attr)
        perf_names = ['accuracy','recall','precision','f1score','mcc','spd','aod','eod',
                      'spd2','aod2','eod2','tpr_p','tpr_u','tnr_p','tnr_u','acc_p','acc_u']
        with open(f"{drive_root}/cot_opt_TinyLlama_{ds}_{attr}_{ver}.txt","w") as fout:
            for name,val in zip(perf_names,res):
                fout.write(f"{name}\t{val:.6f}\n")

        print(f"✓ Done {ds}-{attr}-{ver} | Test Acc={res[0]:.3f}")
        del mdl; torch.cuda.empty_cache(); gc.collect()
